In [ ]:
import tensorflow as tf
print(tf.__version__)

# The Sequential model API

***
<a id="coding_tutorial_1"></a>
## Building a Sequential model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax

#### Build a feedforward neural network model

In [ ]:
# Build the Sequential feedforward neural network model

model=Sequential([
    Flatten(input_shape=(28,28)),
    Dense (16, activation='relu'),
    Dense (16, activation='relu'),
    Dense (10, activation='softmax')
])

In [ ]:
# Print the model summary
#model
#model.weights
model.summary()

***
<a id="coding_tutorial_2"></a>
## Convolutional and pooling layers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

#### Build a convolutional neural network model

In [ ]:
# Build the Sequential convolutional neural network model


model=Sequential([
#    Conv2D(16,(3,3), padding='SAME', strides=2, activation='relu', input_shape=(28,28,1)),
    Conv2D(16,(3,3), 
           padding='SAME', 
           strides=2, 
           activation='relu', 
           input_shape=(28,28,1),
           data_format='channels_last'
          
          ),

    #1 in (28,28,1) indicates the number of channels in the input
    MaxPooling2D((3,3)),
    Flatten(), 
    #Dense (64, activation='relu'), #dense layers require number of parameters
    Dense (10, activation='softmax')
])


model2=Sequential([
#    Conv2D(16,(3,3), padding='SAME', strides=2, activation='relu', input_shape=(28,28,1)),
    Conv2D(16,(3,3), 
           padding='SAME', 
           strides=2, 
           activation='relu', 
           input_shape=(1,28,28), ####updated value!!!!
           data_format='channels_first'
          
          ),

    #1 in (28,28,1) indicates the number of channels in the input
    MaxPooling2D((3,3), 
                 data_format='channels_first'),
    Flatten(), 
    #Dense (64, activation='relu'), #dense layers require number of parameters
    Dense (10, activation='softmax')
])

In [ ]:
# Print the model summary

model.summary()
model2.summary()

***
<a id="coding_tutorial_3"></a>
## The compile method

#### Compile the model

In [ ]:
# Define the model optimizer, loss function and metrics
model.compile(
    optimizer='sgd', #adam, resprop, adadelta
    loss='sparse_categorical_crossentropy', 
        ##mean_squared_error
        ##categorical_crossentropy 
        ##sparse_categorical_crossentropy
        ##binary_crossentropy
    metrics=['accuracy','mae']
)

model2.compile(
    optimizer=tf.keras.optimizers.SGD(),#'sgd', #same, but allows control over params
    loss=tf.keras.losses.BinaryCrossentropy(),#'binary_crossentropy', #mean_squared_error categorical_crossentropy
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.MeanAbsoluteError()]#['accuracy','mae']
)

In [ ]:
# Print the resulting model attributes
print(model.optimizer)
print(model.loss_functions)
print(model.metrics)
#dir(model)

In [ ]:
model3=Sequential([
    Conv2D(16,(3,3), padding='SAME',strides=2,activation='relu',input_shape=(28,28,1),data_format='channels_last'),

    MaxPooling2D((3,3)),
    Flatten(), 
    #Dense (64, activation='relu'), #dense layers require number of parameters
    Dense (10, activation='linear') ####ATTENTION: LINEAR!!! technically no activation.
])

opt=tf.keras.optimizers.SGD(learning_rate=0.007)

model3.compile(
    optimizer=opt,
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        # loss is computed directly from the last dense layer,
        # which technically has no activation.
        # Loss is compuyted on the data that is not passed to sigmoid 
        # Better NUMERICAL STABILTY
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.7), tf.keras.metrics.MeanAbsoluteError()]
)

***
<a id="coding_tutorial_4"></a>
## The fit method

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#### Load the data

In [ ]:
# Load the Fashion-MNIST dataset

fashion_mnist_data = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist_data.load_data()

In [ ]:
# Print the shape of the training data

print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
# Define the labels

labels = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

In [ ]:
# Rescale the image values so that they lie in between 0 and 1.
"""
m=0
for i in train_images:
    for j in i:
        for k in j:
            if m < k: m=k
print(m)
"""
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Display one of the images
i = 0
img = train_images[i,:,:]
%matplotlib inline
#help(plt)
plt.imshow(img)
plt.show()
print(f"label: {labels[train_labels[i]]}")

#### Fit the model

In [ ]:
# Fit the model
#model.fit(train_images, train_labels, epochs=2,batch_size=256)
##above line will give an error, since keras does not see a channel dimention in the data
##to slve, add a dimention to the training data: [...,np.newaxis]:
model.fit(train_images[...,np.newaxis], train_labels, epochs=2,batch_size=256)

#### Plot training history

In [ ]:
# Load the history into a pandas Dataframe

history = model.fit(train_images[...,np.newaxis], train_labels, epochs=10,batch_size=256, verbose=2)

In [ ]:
# Make a plot for the loss

df = pd.DataFrame(history.history)
df.head()

In [ ]:
# Make a plot for the accuracy
%matplotlib inline
lossplot = df.plot(y="loss", title="Loss vs Epochs", legend=False)
lossplot.set(xLabel="Epochs", yLabel="Loss")

In [ ]:
# Make a plot for the additional metric
lossplot = df.plot(y="loss", title="Loss vs Epochs", legend=False)
lossplot.set(xLabel="Epochs", yLabel="Loss")

lossplot = df.plot(y="mae", title="Loss vs Epochs", legend=False)
lossplot.set(xLabel="Epochs", yLabel="MAE")
#help(df.plot)

lossplot = df.plot(y="accuracy", title="Loss vs Epochs", legend=False)
lossplot.set(xLabel="Epochs", yLabel="accuracy")


#help(model.compile)

In [ ]:
#dir(history)
history.params

***
<a id="coding_tutorial_5"></a>
## The evaluate and predict methods

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#### Evaluate the model on the test set

In [ ]:
# Evaluate the model

r = model.evaluate(test_images[...,np.newaxis], test_labels, verbose=2)
#test_loss, test_acc, test_mae = model.evaluate(test_images[...,np.newaxis], test_labels, verbose=2)

In [ ]:
print(r)

#### Make predictions from the model

In [ ]:
# Choose a random test image

random_inx = np.random.choice(test_images.shape[0])

test_image = test_images[random_inx]
plt.imshow(test_image)
plt.show()
print(f"Label: {labels[test_labels[random_inx]]}")

In [ ]:
# Get the model predictions

prediction = model.predict(test_image[np.newaxis, ... , np.newaxis])
print(labels[np.argmax(prediction)])